# CONSTRUINDO PIPELINE

In [9]:
import numpy as np
import pandas as pd
import geopandas as gpd
import warnings
from grid import *
from statsmodels.tsa.statespace.sarimax import SARIMAX

warnings.filterwarnings('ignore')

# Função para carregar e preparar dados (ajuste com seus dados)
def load_and_prepare_data(file_path):
    cols_wanted = ['data_hora_fato', 'natureza_descricao', 'numero_latitude', 'numero_longitude']
    df = pd.read_csv(file_path, usecols=cols_wanted)
    df['data_hora_fato'] = pd.to_datetime(df['data_hora_fato'], errors='coerce')
    df=df.dropna()
    df = df[df['data_hora_fato'].dt.year == 2024]
    df= df[df['natureza_descricao']=='FURTO']
    # Ordenar o DataFrame por data
    df = df.sort_values(by='data_hora_fato')
    
    # Filtrar os primeiros 7 meses
    start_date = df['data_hora_fato'].min()
    end_date = start_date + pd.DateOffset(months=7)
    df = df[(df['data_hora_fato'] >= start_date) & (df['data_hora_fato'] < end_date)]

    return df

# Função para agregar os dados por célula e por dia
def aggregate_data(data, grid):
    """
    Agrega os dados por célula do grid e por dia.
    """
    # Verificar se a coluna de coordenadas existe
    if 'numero_longitude' not in data.columns or 'numero_latitude' not in data.columns:
        raise ValueError("Colunas 'numero_longitude' e 'numero_latitude' estão ausentes no DataFrame.")
    
    # Converter a coluna de data para datetime
    data['data_hora_fato'] = pd.to_datetime(data['data_hora_fato'], errors='coerce')
    
    # Criar geometria com base em latitude e longitude
    data['geometry'] = gpd.points_from_xy(data['numero_longitude'], data['numero_latitude'])
    data_gdf = gpd.GeoDataFrame(data, geometry='geometry', crs='EPSG:4326')

    # Verificar se o CRS do grid está alinhado com os pontos
    if grid.crs.to_string() != data_gdf.crs.to_string():
        grid = grid.to_crs(data_gdf.crs)
    
    # Realizar junção espacial para associar os pontos às células do grid
    joined = gpd.sjoin(data_gdf, grid, how='inner', predicate='intersects')
    
    # Verificar se a junção retornou resultados
    if joined.empty:
        print("A junção espacial retornou um DataFrame vazio.")
        return pd.DataFrame()

    # Garantir que o índice seja do tipo DatetimeIndex
    joined['data_hora_fato'] = pd.to_datetime(joined['data_hora_fato'])
    joined.set_index('data_hora_fato', inplace=True)
    
    # Agregar os dados por célula e por dia
    grouped = joined.groupby(['cell', pd.Grouper(freq='D')]).size()

    # Verificar se a agregação produziu resultados
    if grouped.empty:
        print("A agregação não produziu resultados.")
        return pd.DataFrame()
    
    # Transformar os resultados em um DataFrame
    aggregated_data = grouped.unstack(level=0, fill_value=0)
    return aggregated_data

# Função para dividir dados em treino e teste
def train_test_split_data(data, test_size=7):
    train_data = data.iloc[:-test_size]
    test_data = data.iloc[-test_size:]
    return train_data, test_data

# Função para construir e treinar o modelo SARIMA
def train_and_predict_sarima(train_data, steps=7):
    predictions = []
    real_values = []
    
    for cell in train_data.columns:
        ts = train_data[cell]
        
        # Verifique se o índice é um DatetimeIndex
        if not isinstance(ts.index, pd.DatetimeIndex):
            ts.index = pd.to_datetime(ts.index)

        if ts.sum() == 0:  # Se a série é composta apenas de zeros, prever zero
            predictions.append([0] * steps)
            real_values.append([0] * steps)
            continue

        # Treinar o modelo SARIMA
        model = SARIMAX(ts, order=(1, 1, 1), seasonal_order=(1, 1, 1, 7))
        model_fit = model.fit(disp=False)
        
        # Fazer predições para os próximos 7 dias
        forecast = model_fit.forecast(steps=steps)

        # Garantir que a previsão tenha um índice de data
        forecast.index = pd.date_range(start=ts.index[-1] + pd.Timedelta(days=1), periods=steps)
        
        predictions.append(forecast)
        real_values.append(ts[-steps:].values)
    
    # Converter para DataFrame
    predictions_df = pd.DataFrame(predictions).T
    real_values_df = pd.DataFrame(real_values).T
    predictions_df.columns = train_data.columns
    real_values_df.columns = train_data.columns
    
    return predictions_df, real_values_df



In [10]:
grid_size = 800
municipalities = ["BELO HORIZONTE"]
grid = create_grid(grid_size, municipalities)

# Carregamento e preparação dos dados
file_path = '/home/amarante/Jobs/PM/dataset/dados_BH.csv'
data = load_and_prepare_data(file_path)

# Agregação dos dados no grid
aggregated_data = aggregate_data(data, grid)
# Ajustar o DataFrame para ter todas as células do grid, preenchendo com zeros onde necessário
aggregated_data = aggregated_data.reindex(columns=grid['cell'], fill_value=0)
# Separar treino e teste
train_data, test_data = train_test_split_data(aggregated_data, test_size=30)
predictions, real_values = train_and_predict_sarima(train_data, steps=30)

100%|██████████| 30/30 [00:00<00:00, 3064.74it/s]


# Métricas

In [7]:
hit_rate_percentage = 0.1

def hit_rate( grid_predicted, grid_test, ncells=0):
    if not ncells:
        num_cells = int(len(grid_predicted) * (hit_rate_percentage))
        top_cells = grid_predicted.nlargest(num_cells, "count")
    else:
        top_cells = grid_predicted.nlargest(ncells, "count")

    hit_rate = (
        grid_test[grid_test["cell"].isin(top_cells["cell"])]["count"].sum()
        / grid_test["count"].sum()
    )

    return hit_rate

def average_logarithmic_score( grid_predicted, grid_test):
    val = grid_predicted["count"].to_numpy()[
        grid_test[grid_test["count"] > 0]["cell"].values
    ]
    val[val < 0] = 0
    zeros = (val == 0).sum()
    val[val == 0] = 1
    return np.mean(np.log(val)), zeros

def mean_squared_error( grid_predicted, grid_test):
    return np.mean((grid_predicted["count"] - grid_test["count"]) ** 2)

def pai( grid_predicted, grid_test, ncells=0):

    hr = hit_rate(grid_predicted, grid_test, ncells)

    num_cells = int(len(grid_predicted) * (hit_rate_percentage))
    grid_area = grid_size * grid_size

    a = num_cells * grid_area
    A = len(grid_predicted) * grid_area

    return (hr / (a/A))

def pei( grid_predicted, grid_test):

    
    num_cells = grid_test[grid_test["count"] > 0]["cell"].nunique()
    max_cells = int(len(grid_predicted) * (hit_rate_percentage))

    num_cells = min(max_cells, num_cells)

    pai_ = pai(grid_predicted, grid_test, ncells=num_cells)

    grid_area = grid_size * grid_size
    a = num_cells * grid_area
    A = len(grid_predicted) * grid_area


    return pai_ / (1/(a/A))

def eval(grid_predicted, grid_test):

    grid_predicted = grid_predicted.sort_values("cell")
    grid_test = grid_test.sort_values("cell")

    metrics = {
        f"HR({hit_rate_percentage*100}%)": hit_rate(grid_predicted, grid_test),
        "ALS": average_logarithmic_score(grid_predicted, grid_test)[0],
        "ALS_zeros": average_logarithmic_score(grid_predicted, grid_test)[1],
        "MSE": mean_squared_error(grid_predicted, grid_test),
        "PAI": pai(grid_predicted, grid_test),
        "PEI": pei(grid_predicted, grid_test)
    }

    return metrics

# Resultados

In [11]:
results = []
for i in range(30):
    predictions_df = pd.DataFrame({"count":predictions.iloc[i]/predictions.iloc[i].sum()}).reset_index()
    real_values_df = pd.DataFrame({"count":real_values.iloc[i]/real_values.iloc[i].sum()}).reset_index()
    
    res = eval(predictions_df, real_values_df)
    results.append(res)

results_df = pd.DataFrame(results)
results_df.mean()

HR(10.0%)    0.653283
ALS         -5.891289
ALS_zeros    3.366667
MSE          0.000013
PAI          6.532834
PEI          0.564818
dtype: float64